<a href="https://colab.research.google.com/github/badri449/DeepLearning/blob/main/miniproject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
! pip install d2l==0.17.2

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [11]:

class BasicBlock(nn.Module):

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out



class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def project1_model():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [5]:
model=project1_model()

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    # torch.numel() returns number of elements in a tensor

print(count_parameters(model))

11173962


In [ ]:
print(model)

DATA IMPORTING SECTION BELOW ************************

In [8]:

Transform =  transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])
trainingdata = torchvision.datasets.CIFAR10(root='./ data', train=True, download=True, transform=Transform) 
testdata = torchvision.datasets.CIFAR10(root='./data',  train=False, download=True, transform=Transform)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
trainDataLoader = DataLoader(trainingdata, batch_size= 64, shuffle=True)
testDataLoader = DataLoader(testdata, batch_size= 64, shuffle=False)

In [13]:
Loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
save_loss = {'train':[], 'val':[]}
save_acc = {'train':[], 'val':[]}

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_loss_history = []
test_loss_history = []
train_acc_history = [] 
test_acc_history = [] 


for epoch in range(5):
    train_loss = 0.0
    test_loss = 0.0
    correct_points_train = 0 
    correct_points_test = 0 
    for i, data in enumerate(trainDataLoader):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        
        predicted_output = model(images)
        fit = Loss(predicted_output,labels)
        optimizer.zero_grad()
        fit.backward()
        optimizer.step()
        train_loss += fit.item()
        correct_points_train +=  (torch.eq(torch.max(predicted_output, 1)[1],labels).sum()).data.cpu().numpy()
    # for i, data in enumerate(testDataLoader):
    #     with torch.no_grad():
    #         images, labels = data
    #         images = images
    #         labels = labels
    #         predicted_output = model(images)
    #         fit = Loss(predicted_output,labels)
    #         test_loss += fit.item()
    #         correct_points_test +=  (torch.eq(torch.max(predicted_output, 1) [1],labels).sum()).data.cpu().numpy() 
            
    train_loss = train_loss/len(trainDataLoader)
    test_loss = test_loss/len(testDataLoader)
    
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    
    train_acc = correct_points_train/len(trainDataLoader) 
    test_acc = correct_points_test/len(testDataLoader) 
    
    train_acc_history.append(train_acc) 
    test_acc_history.append(test_acc)

    print('Epoch %s, Train loss %s, Test loss %s Train accuracy %s Test accuracy  %s'%(epoch, train_loss, test_loss, train_acc, test_acc))
    #print("Accuracy:", (torch.eq(torch.max(predicted_output, 1)[1],labels).sum()/len(labels)*100).data.cpu().numpy())

Epoch 0, Train loss 1.512190195986682, Test loss 0.0 Train accuracy 28.185421994884912 Test accuracy  0.0


In [29]:
model = models.resnet34(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]